<a href="https://colab.research.google.com/github/visha1Sagar/BRSR-Report-Generation/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma gradio langchain-openai pypdf langchain-experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 20.1 MB/s eta

In [ ]:
import os
import json
from openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI

In [ ]:
from google.colab import userdata


In [ ]:
AZURE_OPENAI_API_KEY = userdata.get('AZURE_OPENAI_API_KEY')
api_version = userdata.get('api_version')
AZURE_ENDPOINT = userdata.get('AZURE_ENDPOINT')

In [ ]:
from langchain_core.prompts import PromptTemplate


In [ ]:
!pip install --quiet pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 65.3 MB/s eta 0:00:00


In [ ]:
import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import pdfplumber
import pandas as pd

In [ ]:


# # Set up your Azure OpenAI API details
# os.environ["OPENAI_API_KEY"] = "YOUR_AZURE_OPENAI_API_KEY"
# api_base = "https://YOUR_RESOURCE_NAME.openai.azure.com/"
# api_version = "2023-03-15-preview"
# model_name = "gpt-4"  # Replace with your deployed GPT-4 model name

llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY  ,
    api_version=api_version,
    azure_endpoint = AZURE_ENDPOINT
    )


In [ ]:
df = pd.read_excel("ghg_data08.xlsx")

In [ ]:
from langchain.agents import AgentType
from langchain_experimental.agents import create_pandas_dataframe_agent


pandas_df_agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,
    handle_parsing_errors=True,
)


/usr/local/lib/python3.10/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [ ]:

response = pandas_df_agent.run("Total electricity consumption (A)")





> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "total_electricity_consumption = df[(df['Category'] == 'Electricity purchase')]['Total_Value'].sum()"}`



Invoking: `python_repl_ast` with `{'query': "total_electricity_consumption = df[(df['Category'] == 'Electricity purchase')]['Total_Value'].sum() \ntotal_electricity_consumption"}`


5436207.0The total electricity consumption (A) is 5,436,207 kWh.

> Finished chain.


In [ ]:
from langchain.prompts import PromptTemplate


# Prompts with templates
template_data_extraction_prompt = PromptTemplate(
    input_variables=["pdf_content"],
    template="""
You are a specialized AI assistant assigned to identify and list specific data points required to complete a section of a Business Responsibility and Sustainability Report (BRSR) template.
Based on the following PDF content, focus on identifying only the essential data items necessary for precise and accurate completion of the template.
Please avoid making assumptions or including irrelevant data points in your response. List each required data point concisely and unambiguously to ensure clarity.

PDF Content:
{pdf_content}
"""
)

csv_data_extraction_prompt = PromptTemplate(
    input_variables=["csv_content", "response", "topic"],
    template="""
As an AI assistant focused on extracting and preparing data from a CSV file, your task is to accurately identify and process specific data fields outlined in the ChatGPT response: {response}.
Begin by locating each of these data fields within the CSV content provided, ensuring they are precisely represented and appropriately transformed as needed. Beyond these predefined fields, scan the content for any additional data relevant to the topic: {topic}.
Ensure all data is clearly quantified and accurately captured to support BRSR reporting requirements.

CSV Content:
{csv_content}
"""
)

report_part_writing_prompt = PromptTemplate(
    input_variables=["data", "topic"],
    template="""
You are an AI assistant tasked with drafting the section "{topic}" of a Business Responsibility and Sustainability Report (BRSR) using the data provided: {data}. Use the attached sample of the "{topic}" section as a general guide for structure, formatting, and tone, ensuring that the draft is both comprehensive and aligned with BRSR standards.

Guidelines for Drafting the Section:

- Adherence to Template Structure: Follow the sequence and structure of the attached sample to maintain alignment with BRSR reporting standards. Include main headings and subheadings as presented in the template but feel free to adapt the content and add context where necessary to fit the new data.
- Quantitative Data in Tables: Present all quantitative data in table format to enhance readability and clarity. Where tables are not feasible, ensure that quantitative details are still clear, precise, and easy to locate.
- Handling Missing Data: If specific data fields are missing from {data}, label them as "Not Available" within the draft to indicate incomplete data transparently.
- Structured Flexibility for Completeness: Use the provided template as a structural foundation, but adapt it to ensure all relevant data is logically organized. Feel free to create additional minor headings or sub-sections if it aids in clarity and does not diverge from the BRSR’s standards.
- Professional and Clear Presentation: Ensure that the report section is presented professionally, with an emphasis on clarity, data accuracy, and alignment with BRSR conventions.

Final Output Expectations: The draft should accurately reflect both the content provided and the intended structure, maintaining all essential headings and a professional tone while integrating the data in a clear, organized, and reader-friendly manner.
"""
)

paraphrase_text_part = PromptTemplate(
    input_variables=["text", "message", "report"],
    template="""
You are an AI assistant assigned to paraphrase part of reports, given context : {report},
1. Find {text} in it and
2. Paraphrase it given user's message : {message}

The final output should be paraphrased text based on given user's message .
"""
)

In [ ]:


# File processing functions
def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
    return text

def extract_text_from_csv(file_path):
    filetype = file_path.split(".")[-1]

    if filetype == "xlsx":
        df = pd.ExcelFile(file_path)
        df_string = ""
        for sheet_name in df.sheet_names:
            df_string += df.parse(sheet_name).to_string(index=False)
        return df_string
    elif filetype == "csv":
        df = pd.read_csv(file_path)
    else:
        return "Couldn't read Spreadsheet - Error - filename :"+file_path
    return df.to_string(index=False)  # Convert to string for prompt input

# Function to call Azure OpenAI with each prompt type
def run_template_data_extraction(file_path):
    try:
        pdf_content = extract_text_from_pdf(file_path)
        prompt = template_data_extraction_prompt.format(pdf_content=pdf_content)
        response = llm.invoke(prompt)
        return response.content
    except Exception as e:
        return f"Error: {e}"

def run_csv_data_extraction(file_path, response_text, topic):
    try:
        csv_content = extract_text_from_csv(file_path)
        prompt = csv_data_extraction_prompt.format(csv_content=csv_content, response=response_text, topic=topic)
        response = llm.invoke(prompt)

        return response.content
    except Exception as e:
        return f"Error: {e}"

def run_report_part_writing(data, topic):
    prompt = report_part_writing_prompt.format(data=data, topic=topic)
    try:
        response = llm.invoke(prompt)
        return response.content
    except Exception as e:
        return f"Error: {e}"





In [ ]:
# Example usage
template_data_response = run_template_data_extraction("brsr.pdf")


In [ ]:
print(template_data_response)

### Essential Data Points for BRSR Completion

1. **Total Energy Consumption**:
   - Total electricity consumption (FY 2022-23, FY 2021-22)
   - Total fuel consumption (FY 2022-23, FY 2021-22)
   - Energy consumption through other sources (Steam) (FY 2022-23, FY 2021-22)
   - Total energy consumption (A+B+C) (FY 2022-23, FY 2021-22)
   - Energy intensity per rupee of turnover (Total energy consumption/ turnover in Million)

2. **Designated Consumers**:
   - Whether the entity has designated consumers under the PAT Scheme (Y/N)
   - Targets achieved under the PAT scheme (Yes/No)
   - Remedial actions taken if targets were not achieved.

3. **Water Withdrawal Details**:
   - Water withdrawal by source (in kilolitres) (Surface water, Groundwater, Third party water, Seawater/Desalinated Water, Others) (FY 2022-23, FY 2021-22)
   - Total volume of water withdrawal (in kilolitres) (i + ii + iii + iv + v) (FY 2022-23, FY 2021-22)
   - Total volume of water consumption (in kilolitres) (FY 2022

In [ ]:
csv_data_extraction_response = run_csv_data_extraction("ghg_data08.xlsx",template_data_response , "Principle 6 of Section C")


In [ ]:
report_response = run_report_part_writing(csv_data_extraction_response, "Principle 6 of Section C")

In [ ]:
print(report_response)

# Principle 6: Environmental Management

This section outlines our commitment to environmental sustainability, detailing our energy consumption, water usage, air emissions, greenhouse gas emissions, and waste management practices. We strive to minimize our environmental footprint through responsible resource management and compliance with applicable regulations.

## 1. Total Energy Consumption

| Category                              | FY 2022-23 (kWh) | FY 2021-22 (kWh) |
|---------------------------------------|------------------|------------------|
| Total Electricity Consumption          | 1,275,247        | 1,246,798        |
| Total Fuel Consumption (Natural Gas)  | 790,508          | 782,260          |
| **Total Energy Consumption (A + B)**  | **2,065,755**    | **2,029,058**    |

- **Energy Intensity per Rupee of Turnover:** Not Available

## 2. Designated Consumers

- **Whether the entity has designated consumers under the PAT Scheme:** Yes
- **Targets achieved under the PAT 